<a href="https://colab.research.google.com/github/eldiablo-data/tensorflow2learning/blob/master/notebooks/Fashion_MNIST_with_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import numpy as np
tf.__version__

In [ ]:
from tensorflow.keras.layers import Input, Conv2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Model

# Fashion MNIST
This dataset is more complex than the regular MNIST dataset. It contains 10 classes of clothing and apparel object images in gray scale of size 28 x 28. We load this dataset from Keras similar to MNIST.

In [ ]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.fashion_mnist.load_data()
x_train_scaled = x_train / 255
x_test_scaled = x_test /255
print("x_train.shape:",x_train.shape)

The input is a 28 x 28 grayscale image. For convolution, colored images are expected (height, width and colors as 3 dimensions) and  it requires an additional dimension in the input. Hence, the scalar color is represented as a 1D vector.

In [ ]:
x_train_scaled_cnn = np.expand_dims(x_train_scaled, -1)
x_test_scaled_cnn = np.expand_dims(x_test_scaled,-1)
print("x_train_scaled_cnn.shape:",x_train_scaled_cnn.shape)

In [ ]:
k = len(set(y_train))
print("Number of classes: ", k)

## Modelling

In [ ]:
i = Input(shape=x_train_scaled_cnn[0].shape)
x = Conv2D(filters=32,kernel_size=(3,3),strides=2, activation='relu')(i)
x = Conv2D(filters=64,kernel_size=(3,3),strides=2, activation='relu')(x)
x = Conv2D(filters=128,kernel_size=(3,3),strides=2, activation='relu')(x)
x = Flatten()(x)
x = Dropout(0.2)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
x= Dense(k, activation='softmax')(x)
model = Model(i,x)

We add a 2D convolution layer of size 32 with kernel size 3x3. This is a fine layer looking into patterns and strokes. In the next layer, we will increase the filters to look for larger patterns like, say buttons and stitches. In the final layer, we will increase the filter to even larger patterns such as collars and sleeves. 
Strides make the model skip a certain number of neighbouring pixels to save computation times. Usually immediate neighbours tend to have similary color values. 

We have 3 convolution layers. After that, we switch to regular Dense layer for classification. 

In [ ]:
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics='accuracy')
r = model.fit(x_train_scaled_cnn,y_train,validation_data=(x_test_scaled_cnn,y_test),epochs = 15)

In [ ]:
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(y= r.history['val_accuracy'],name="Validation Accuracy"))
fig.add_trace(go.Scatter(y=r.history['accuracy'],name="Training Accuracy"))
fig.update_layout(title="Training Progress - Accuracy", xaxis_title="Epochs",yaxis_title="Accuracy %" )
fig.show()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=r.history['val_loss'],name="Validation Loss"))
fig.add_trace(go.Scatter(y=r.history['loss'],name="Training Loss"))
fig.update_layout(title = "Training Progress - Loss",xaxis_title="Epochs",yaxis_title="Loss %")
fig.show()

In [ ]:
y_pred = model.predict(x_test_scaled_cnn).argmax(axis=1)

In [ ]:
# Label mapping
labels = '''T-shirt/top
Trouser
Pullover
Dress
Coat
Sandal
Shirt
Sneaker
Bag
Ankle boot'''.split("\n")

In [ ]:
from pandas import crosstab as tab
confusion=tab(index=y_test,columns=y_pred,colnames=['Predicted Lable'],rownames=['True Lable'])
confusion

In [ ]:
import plotly.express as px
px.imshow(confusion,zmax=120,title='Predicted vs True Labels')

In [ ]:
misclassified_idx = np.where(y_pred != y_test)[0]

In [ ]:
# Show some misclassified examples
i = np.random.choice(misclassified_idx)
px.imshow(x_test[i].reshape(28,28),color_continuous_scale="blues",
          title="True label: %s Predicted: %s" % (labels[y_test[i]], labels[y_pred[i]]))